In [827]:
import json
import numpy as np
from collections import defaultdict
import math

In [828]:
with open('data/slots/time_slots.json', 'r') as f:
    slots_data = json.load(f)

with open('data/obrony/obrony.json', 'r') as f:
    defences_data = json.load(f)

with open('data/people/przewodniczacy.json', 'r') as f:
    chairmans = json.load(f)

with open('data/people/workers.json', 'r') as f:
    members = json.load(f)

with open('data/examples/92.json', 'r') as f:
    availability_data = json.load(f)

In [829]:
availability = {}
for person, slot_index in availability_data:
    if (slot_index + 1) % 6 == 0:
        availability[person] = [slot_index - 1, slot_index]
    else:
        availability[person] = [slot_index, slot_index + 1]

In [830]:
len(availability)

35

In [831]:
# availability = {}
# for person, slot_index in availability_data:
#     if (slot_index + 1) % 6 == 0:
#         availability[person] = [slot_index - 2, slot_index - 1, slot_index]
#     elif (slot_index) % 6 == 0:
#         availability[person] = [slot_index, slot_index + 1, slot_index + 2]
#     else:
#         availability[person] = [slot_index - 1, slot_index, slot_index + 1]

In [832]:
from Defence_slots import Defence, Slot

defences = [Defence(person_data["student"], person_data["promotor"], person_data["recenzent"]) for person_data in defences_data]

slots = [Slot(index, time) for index, time in slots_data.items()]

In [833]:
import math
from State_action import State

class Environment():
    def __init__(self, defences, slots, chairmans, members, availability):
        self.defences = defences
        self.slots = slots
        self.chairmans = chairmans
        self.members = members
        self.availability = availability

    def get_initial_state(self):
        # if /6 are equal it means that it's the same day
        promoter = self.defences[0].promoter
        reviewer = self.defences[0].reviewer
        available_slots_promoter = self.availability[promoter]
        available_slots_reviewer = self.availability[reviewer]
        available_slots = available_slots_promoter + available_slots_reviewer
        available_chairmans = []
        available_members = []
        for slot in available_slots:
            for chairman in self.chairmans:
                if math.floor(slot / 6) == math.floor(self.availability[chairman][0] / 6):
                    available_chairmans.append(chairman)
            for member in self.members:
                if slot in self.availability[member]:
                    available_members.append(member)

        available_chairmans = list(set(available_chairmans))
        available_members = list(set(available_members))

        available_chairmans = list(filter(lambda x: x!= promoter, available_chairmans))
        available_chairmans = list(filter(lambda x: x!= reviewer, available_chairmans))
        available_members = list(filter(lambda x: x!= promoter, available_members))
        available_members = list(filter(lambda x: x!= reviewer, available_members))

        return State(self.defences[0], available_slots, available_chairmans, available_members)

    def get_reward(self, defence, slot, chairman, member):
        if chairman in self.chairmans and member in self.members and \
           slot in self.availability[chairman] and slot in self.availability[member] and \
            slot in self.availability[defence.promoter] and slot in self.availability[defence.reviewer]:
            return 1
        else:
            return -1

    def take_action(self, state, action):
        # Unpack state and action
        defence = state.defence
        available_slots = state.available_slots
        available_chairmans = state.available_chairmans
        available_members = state.available_members
        # defence, available_slots, available_chairmans, available_members = state

        slot = action.slot
        chairman = action.chairman
        member = action.member
        # slot, chairman, member = action

        # Check if the action is valid
        if chairman not in available_chairmans or member not in available_members or slot not in available_slots:
            return state, -1

        # Assign chairman, member and slot
        # self.chairman = chairman
        # self.member = member
        # self.slot = slot

        # Update lists of available chairmans and members
        available_chairmans.remove(chairman)
        available_members.remove(member)
        available_slots.remove(slot)

        # Move to the next defence and slot
        if self.defences.index(defence) < len(self.defences) - 1:
            next_defence = self.defences[self.defences.index(defence) + 1]

            # Update lists of available chairmans and members
            promoter = next_defence.promoter
            reviewer = next_defence.reviewer
            available_slots_promoter = self.availability[promoter]
            available_slots_reviewer = self.availability[reviewer]
            available_slots = available_slots_promoter + available_slots_reviewer
            available_chairmans = []
            available_members = []

            for slot in available_slots:
                for chairman in self.chairmans:
                    if math.floor(slot / 6) == math.floor(self.availability[chairman][0] / 6):
                        available_chairmans.append(chairman)
                for member in self.members:
                    if slot in self.availability[member]:
                        available_members.append(member)

            available_chairmans = list(set(available_chairmans))
            available_members = list(set(available_members))

            available_chairmans = list(filter(lambda x: x!= promoter, available_chairmans))
            available_chairmans = list(filter(lambda x: x!= reviewer, available_chairmans))
            available_members = list(filter(lambda x: x!= promoter, available_members))
            available_members = list(filter(lambda x: x!= reviewer, available_members))


            # Update state
            next_state = State(next_defence, available_slots, available_chairmans, available_members)

            # Get reward
            reward = self.get_reward(defence, slot, chairman, member)
        else:
            next_defence = None
            next_state = None
            reward = None
            return next_state, reward

        return next_state, reward

In [834]:
# from Environment import Environment
from State_action import Action

Q_table = defaultdict(float)

# Set learning parameters
alpha = 0.5
gamma = 0.9
epsilon = 0.1 # for exploration-exploitation tradeoff
num_episodes = 1000

env = Environment(defences, slots, chairmans, members, availability)


In [835]:
print(defences[0])

Defence exam: student: Ernest Pietuszyński, promoter: dr hab. T.Gambin, prof. PW, reviewer: prof. dr hab. P.Rokita


In [836]:
# number_of_slots = 30

for episode in range(num_episodes):
    # slots = [i for i in range(number_of_slots)]
    # Initialize state
    # state - defence, available_slots, available_chairmans, available_members
    state = env.get_initial_state()

    for _ in range(100):  # assume each episode has 100 steps
        defence = state.defence
        available_slots = state.available_slots
        available_chairmans = state.available_chairmans
        available_members = state.available_members

        # print(f'{defence}')
        # print(f'available_slots: {available_slots}')
        # print(f'available_chairmans: {available_chairmans}')
        # print(f'available_members: {available_members}')

        # print(f'EPISODE: {episode}')

        if np.random.rand() < epsilon or (state, 'Exploit') not in Q_table:
            # Exploration: choose random action
            if not available_chairmans or not available_members or not available_slots:
                print(f'No members or chairmans or slots available')
                break
            slot = np.random.choice(available_slots)
            chairman = np.random.choice(available_chairmans)
            member = np.random.choice(available_members)
        else:
            # Exploitation: choose best action based on current estimate
            action = Q_table[(state, 'Exploit')]

        action = Action(slot, chairman, member)

        # Take action
        next_state, reward = env.take_action(state, action)

        if next_state is None or reward is None:
            print('Episode finished.')
            break

        next_defence = next_state.defence
        next_available_slots = next_state.available_slots
        next_available_chairmans = next_state.available_chairmans
        next_available_members = next_state.available_members

        # Update Q-table
        
        old_value = Q_table[(state, action)] if Q_table[(state, action)] else 0
        if available_chairmans and available_members and available_slots:
            max_next_value = 0
            for member in available_members:
                for chairman in available_chairmans:
                    for slot in available_slots:
                            new_action = Action(slot, chairman, member)
                            max_next_value = max(max_next_value, Q_table.get((next_state, new_action), 0))
        else:
            max_next_value = 0

        Q_table[(state, action)] = (1 - alpha) * old_value + alpha * (reward + gamma * max_next_value)

        if next_state.defence is None:
            break

        state = next_state

Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode finished.
Episode fi

In [837]:
print(len(Q_table.items()))
for item in Q_table.items():
    print(item[0][0])
    print("---")
    print(item[0][1])
    print("---------------------------------------")


1851
State: 
 	 Defence exam: student: Ernest Pietuszyński, promoter: dr hab. T.Gambin, prof. PW, reviewer: prof. dr hab. P.Rokita
	 available_slots: [26, 9, 10]
	 available_chairmans: ['dr hab. P.Wawrzyński', 'dr hab. K.Cabaj, prof. PW']
	 available_members: ['dr inż. P.Andruszkiewicz', 'doc. dr inż. R.Podraza', 'dr inż. J.Wytrębowicz', 'dr hab. P.Wawrzyński', 'dr inż. J.Raczkowski', 'dr inż. K.Radlak', 'prof. dr hab. J.Mulawka', 'dr inż. K.Żbikowski', 'dr hab. P.Gawrysiak, prof. PW']
---
Action: 
 	 Picked slot: 25 
 	 Picked chairman: dr hab. P.Gawrysiak, prof. PW 
 	 Picked_member: dr inż. J.Koperwas
---------------------------------------
State: 
 	 Defence exam: student: Mariusz Rzehorzek, promoter: dr inż. P.Zawistowski, reviewer: dr inż. G.Blinowski
	 available_slots: [20, 14, 15]
	 available_chairmans: ['dr hab. R.Nowak, prof. PW']
	 available_members: ['dr hab. W.Zabołotny, prof. PW', 'dr inż. D.Ryżko', 'dr inż. T.Śliwiński', 'mgr inż. W.Wysota']
---
Action: 
 	 Picked slot: 

In [838]:
def actions(state):
    available_slots = state.available_slots
    available_chairmans = state.available_chairmans
    available_members = state.available_members
    possible_actions = [Action(slot, chairman, member) for slot in available_slots for chairman in available_chairmans for member in available_members]
    return possible_actions


In [839]:
state = env.get_initial_state()
schedule = []
count = 0
for i in range(len(defences)):
    # print(i)
    # Get the list of actions
    available_actions = actions(state)

    # Check if the list of actions is empty
    if not available_actions:
        # Break the loop if there are no available actions
        count += 1
        print('No available actions for current state.')
        continue

    # Choose best action according to Q-table
    best_action = None
    best_q_value = -float('inf')
    for action in available_actions:
        if Q_table[(state, action)] > best_q_value:
            best_action = action
            best_q_value = Q_table[(state, action)]
    
    # If best_action is still None, assign a random action
    if best_action is None:
        best_action = np.random.choice(available_actions)
    
    # Perform action and observe new state
    state, _ = env.take_action(state, best_action)

    # Add action to schedule
    schedule.append(best_action)


In [840]:
count

0

In [841]:
for s in schedule:
    print(s)

Action: 
 	 Picked slot: 25 
 	 Picked chairman: dr hab. P.Wawrzyński 
 	 Picked_member: dr inż. P.Andruszkiewicz
Action: 
 	 Picked slot: 19 
 	 Picked chairman: dr hab. R.Nowak, prof. PW 
 	 Picked_member: dr hab. W.Zabołotny, prof. PW
Action: 
 	 Picked slot: 22 
 	 Picked chairman: dr hab. P.Gawrysiak, prof. PW 
 	 Picked_member: dr inż. Ł.Skonieczny
Action: 
 	 Picked slot: 16 
 	 Picked chairman: dr hab. W.Zabołotny, prof. PW 
 	 Picked_member: dr hab. W.Zabołotny, prof. PW
Action: 
 	 Picked slot: 26 
 	 Picked chairman: dr hab. P.Gawrysiak, prof. PW 
 	 Picked_member: dr hab. T.Gambin, prof. PW
Action: 
 	 Picked slot: 2 
 	 Picked chairman: dr hab. P.Gawrysiak, prof. PW 
 	 Picked_member: dr hab. T.Gambin, prof. PW
Action: 
 	 Picked slot: 4 
 	 Picked chairman: dr hab. I.Blumke 
 	 Picked_member: dr inż. P.Zawistowski
Action: 
 	 Picked slot: 2 
 	 Picked chairman: dr hab. P.Wawrzyński 
 	 Picked_member: doc. dr inż. R.Podraza
Action: 
 	 Picked slot: 22 
 	 Picked chairman: 

In [842]:
import pandas as pd

# Convert schedule to a list of dictionaries
schedule_dicts = [action.__dict__ for action in schedule]

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(schedule_dicts)

# Save DataFrame to a JSON file
df.to_json('schedule.json', orient='records', lines=True)
